# Series Analysis Using AMS

In this notebook, we will show how to use AMS to do a series analysis, for example, a series of DCOPF with different load levels.

In [1]:
import numpy as np

import ams

In [2]:
!ams misc --version

Python   3.12.0
ams      1.0.9
andes    1.9.3
numpy    2.0.2
cvxpy    1.6.0
solvers  CLARABEL, ECOS, ECOS_BB,
         GUROBI, HIGHS, MOSEK, OSQP,
         PIQP, SCIP, SCIPY, SCS


In [3]:
ams.config_logger(stream_level=30)

Typically, it is faster to read a JSON case than XLSX.

In [4]:
sp = ams.load("./Texas7k.json",
              setup=True, no_output=True)

For example, there are 4 snapshots DCOPF need to be solved.

In [5]:
lfs = np.array([1.0, 1.05, 1.1, 1.03])

In [6]:
# store the original load values
pq_idx = sp.PQ.idx.v
p0 = sp.PQ.get(src='p0', attr='v', idx=pq_idx).copy()

# initialize the DCOPF routine
sp.DCOPF.init()

for i, lf in enumerate(lfs):
    # update load values with the load factor
    sp.PQ.set(src='p0', attr='v', idx=pq_idx, value=lf * p0)
    # update the corresponding routine parameters
    sp.DCOPF.update('pd')
    # solve it
    sp.DCOPF.run(solver='PIQP', ignore_dpp=True)
    print(f"Snapshot {i+1}: Cost={sp.DCOPF.obj.v:.4f}, Gen={sp.DCOPF.pg.v.sum():.4f}")

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> solved as optimal in 0.1166 seconds, converged in 19 iterations with PIQP.
<DCOPF> solved as optimal in 0.1160 seconds, converged in 19 iterations with PIQP.


Snapshot 1: Cost=1688003.4867, Gen=746.6652
Snapshot 2: Cost=1797494.8097, Gen=783.9984


<DCOPF> solved as optimal in 0.1117 seconds, converged in 19 iterations with PIQP.
<DCOPF> solved as optimal in 0.1106 seconds, converged in 19 iterations with PIQP.


Snapshot 3: Cost=1911816.7208, Gen=821.3317
Snapshot 4: Cost=1753080.6187, Gen=769.0651


You can also update specific load rather than entire load, as discussed in https://ltb.readthedocs.io/projects/ams/en/stable/_examples/ex2.html#Change-Load